In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [6]:
len(age)

23708

In [7]:
# merging ag, gender and image path array into a single dataframe

df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [8]:
df.shape

(23708, 3)

In [9]:
df.head()

age  gender                                    img
0   23       0  23_0_2_20170107213747034.jpg.chip.jpg
1   18       1  18_1_0_20170117133441179.jpg.chip.jpg
2   55       0  55_0_3_20170119191938385.jpg.chip.jpg
3   20       1  20_1_2_20170116163800040.jpg.chip.jpg
4   53       0  53_0_0_20170104212348773.jpg.chip.jpg

In [10]:
# training and test splitting.

train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [11]:
train_df.shape

(20000, 3)

In [12]:
test_df.shape

(3708, 3)

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
def multi_output_generator(generator):
    while True:
        iterator = iter(generator) # Create an iterator from the generator
        x, y = next(iterator)  # Call next on the iterator
        yield x, {'age': y[:, 0], 'gender': y[:, 1]}

train_generator_raw = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(200, 200),
    class_mode='raw',  # Keeps labels as raw numerical values
    batch_size=32
)

test_generator_raw = test_datagen.flow_from_dataframe(
    test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(200, 200),
    class_mode='raw',  # Keeps labels as raw numerical values
    batch_size=32
)

train_generator = multi_output_generator(train_generator_raw)
test_generator = multi_output_generator(test_generator_raw)

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [15]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [16]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output # taking the last layer

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [18]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [19]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [20]:
from keras.utils import plot_model
plot_model(model)